In [1]:
from config import EXPER_PATH, MODEL_PATH
import sys
sys.path.append(MODEL_PATH)
from model.dataset.hpatches import HpatchesDataset

In [2]:
from torch.utils.data import DataLoader

import numpy as np
import torch

from model.hfnet import HFNet

from evaluation.keypoint_detectors import evaluate

import config as cfg

In [3]:
# Load dataset
dataset = HpatchesDataset(alteration='all', make_pairs=True, shuffle=True)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [4]:
EXPER_PATH = EXPER_PATH
MODEL_PATH = MODEL_PATH
models = { 
    'silk':{
        'hfnet-silk lr =3e-1':'weight/hfnet-silk lr =3e-1/last_model.pth', 
        },
}

config_model= {
    'image_channels':1,
    # 'loss_weights': 'uncertainties',
    'loss_weights':{
        'global':1,
        'local':1,
        'detector':1
    },
    'local_head': {
        'scale':2,
        'descriptor_dim': 128,
        'detector_grid': 8,
        'input_channels': 96
    },
    'global_head': {
        'n_clusters': 32,
        'intermediate_proj': 0,
        'dimensionality_reduction': 4096
    }
}

config = {
        'local':{
            'nms_radius':4,
            'detector_threshold': 0.45,
            'num_keypoints': 10000,
        },
        'do_ratio_test': True,
        'correct_match_thresh': 3,
        'correct_H_thresh': 3,
        'num_features': 300,
        # 'do_nms': True,
        # 'nms_thresh': 8,
        'remove_borders': 4,
    }
    


model = HFNet(config_model, width_mult=0.75)

In [5]:
import time

device = torch.device('cpu')
model.to(device)

for k,v in models.items():
    for kk,vv in v.items():
        print(f"{k}: {kk}")
        print(config)
        state_dict = torch.load(MODEL_PATH + vv, weights_only= False ,map_location=device)
        model.load_state_dict(state_dict, strict=False)

        metrics=evaluate(model, dataloader, config)
        

        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(current_time)

        log_file = open('detector.txt', 'a')
        log_file.write('\n[{}]'.format(current_time))
        log_file.write('\n{}: {}'.format(k, kk))
        log_file.write('{}\n'.format(config))


        for km, vm in metrics.items():
                print('{:<25} {:.3f}'.format(km, vm))

                #save in log file
                log_file.write('{:<25} :{:.3f}  '.format(km, vm))
                log_file.write('\n')
        log_file.flush()
        print("-----------------------------------")

       

silk: hfnet-silk lr =3e-1
{'local': {'nms_radius': 4, 'detector_threshold': 0.45, 'num_keypoints': 10000}, 'do_ratio_test': True, 'correct_match_thresh': 3, 'correct_H_thresh': 3, 'num_features': 300, 'remove_borders': 4}


100%|██████████| 580/580 [09:24<00:00,  1.03it/s]

2025-04-22 17:57:05
average_num_keypoints     296.074
localization_error        1.504
repeatability             0.375
mAP                       0.192
-----------------------------------
